In [20]:
%run ./time_series_encoding.py

synthetic_lf().collect().write_parquet("synthetic.parquet")
lf = load_and_preprocess_data("synthetic.parquet")
lf.head().collect()

In [ ]:
lf.columns[1:]

In [ ]:
lf1 = recode_columns(lf)

lf1.head().collect()


1. starting at the lag 2, continuing for `n_prior` lags (so 2 to 2+n_prior inclusive)
2. use the lag 2 value as the target, and the remaining columns as features to predict the lag 2 value
3. using the lag 1 value as the target, and the remaining columns as features, apply the fitted model to the lag 2 to lag 2+n_prior columns to estimate the lag 1 value
4. calculate the MSE of the estimated lag 1 value and the actual lag 1 value 

5. repeat steps 3 and 4, starting at lag 3, fitting the model to the lag 3 to lag 3+n_prior columns, estimating the lag 2 value, calculating the MSE of the estimated lag 2 value and the actual lag 2 value, and so on
6. continue until you have n_folds MSE values
7. return the mean and sd of the MSE values

In [ ]:
get_columns(lf1, "feature1", 5, 2 - 1).collect()

In [ ]:
"""

1. starting at the lag 2, continuing for `n_prior` lags (so 2 to 2+n_prior inclusive)
2. use the lag 2 value as the target, and the remaining columns as features to predict the lag 2 value
3. using the lag 1 value as the target, and the remaining columns as features, apply the fitted model to the lag 2 to lag 2+n_prior columns to estimate the lag 1 value
4. calculate the MSE of the estimated lag 1 value and the actual lag 1 value 

5. repeat steps 3 and 4, starting at lag 3, fitting the model to the lag 3 to lag 3+n_prior columns, estimating the lag 2 value, calculating the MSE of the estimated lag 2 value and the actual lag 2 value, and so on
6. continue until you have n_folds MSE values
7. return the mean and sd of the MSE values
"""


def lagged_forecast(lf: pl.LazyFrame, feature: str, n_folds: int, n_prior: int):
    # for fold in range(1, n_folds + 1):
    fold = 1

    # train model on lag 2 to lag 2+n_prior columns
    y_col = get_columns(lf, feature, n_folds).columns[0]
    x_cols = get_columns(lf, feature, n_folds, fold + 1 - 1).columns[1:]
    print(f"y_cols: {y_col}")
    print(f"x_cols: {x_cols}")
    y = lf.select(y_col).collect().to_numpy().reshape(-1, 1)

    # X0 = lf.select(
    #     [
    #         get_columns(lf, feature, n_folds, fold + i - 1)
    #         for i in range(1, n_prior + 2)
    #     ]
    # ).collect()

    # estimate lag 1 value

    return y


lagged_forecast(lf1, "feature1", 5, 3)

In [ ]:
get_columns(lf1, "feature1", 5, i).columns

In [ ]:
xy = X_y_generator(lf1, "feature1", 5, 2)
count = 0
for x, y in xy:
    count += 1
    print(f"count: {count}")
    print(
        f"\n\n==\nx:\n==\n\n{x.reshape(-1, 1)[:5]}\n\n==\ny:\n==\n\n{y.reshape(-1, 1)[:5]}"
    )

In [ ]:
def extract_features_and_lags(lf: pl.LazyFrame) -> tuple:
    features = [c.split("_")[2] for c in lf.select(cs.starts_with("logit")).columns]

    lags = [
        int(c.split("_")[-1].replace("]", ""))
        for c in lf.select(cs.starts_with("logit")).columns
    ]

    lf = lf.select(
        [pl.col("index")]
        + [
            pl.col(c).alias(f"{f}_{int(l/30)}")
            for c, f, l in zip(
                lf.select(cs.starts_with("logit")).columns, features, lags
            )
        ]
    )
    return lf, features, lags


df, feats, lags = extract_features_and_lags(lf)
df.head().collect()